In [147]:
import torch
import string
import datasets

from torch import nn
import torch.nn.functional as F

In [66]:
ds = datasets.load_dataset("ptb_text_only")

DatasetDict({
    train: Dataset({
        features: ['sentence'],
        num_rows: 42068
    })
    test: Dataset({
        features: ['sentence'],
        num_rows: 3761
    })
    validation: Dataset({
        features: ['sentence'],
        num_rows: 3370
    })
})

In [91]:
itos = {i:j for i,j in enumerate(" "+"."+","+string.ascii_lowercase)}
stoi = {j:i for i,j in itos.items()}

29

In [70]:
text = """ 
Never forget what you are, for surely the world will not. Make it your strength. Then it can never be your weakness. Armour yourself in it, and it will never be used to hurt you.
""".lower().strip()

dataset = [(ch1, ch2) for ch1,ch2 in zip(text, text[1:])]
X = torch.tensor([stoi[i[0]] for i in dataset])
Y = torch.tensor([stoi[i[1]] for i in dataset])

In [83]:
# hyperparams
embed_dim = 2
hidden_size = 2

In [152]:
class RNN(nn.Module):
    def __init__(self, embed_dim=2, vocab_size=29, hidden_size=10):
        super().__init__()
        # embedding
        self.C = torch.rand((len(stoi), hidden_size))

        # intial layer
        # self.W_xi = torch.rand(embed_dim, hidden_size)
        # self.b_xi = torch.rand(hidden_size)

        # weights for reccurent loop
        self.W_xh = torch.rand(hidden_size, hidden_size)
        self.b_xh = torch.rand(hidden_size)

        # hidden state
        self.h = torch.rand(hidden_size)
        self.W_hh = torch.rand(hidden_size, hidden_size)

        # classification head
        self.W_hy = torch.rand(hidden_size, vocab_size)
        self.b_hy = torch.rand(vocab_size)

        self.params = [self.W_xh, self.b_xh, self.W_hh, self.W_hy, self.b_hy]
        
    def step(self, x):

        h_out = self.h @ self.W_hh
        x_out = x @ self.W_xh + self.b_xh

        # this is a new hidden state
        return torch.tanh(x_out+h_out)

    def forward(self, x):
        # embed
        x = self.C[x]
        # x = torch.tanh(x @ self.W_xi + self.b_xi)
        
        self.h = self.step(x)

        return self.h @ self.W_hy + self.b_hy

rnn = RNN()

In [153]:
logits = rnn(X[1])
probas = F.softmax(logits, dim=-1)
_, indx = torch.topk(probas, 1)
itos[indx.item()]

'w'

In [136]:
# my rnn
# embeddings
torch.manual_seed(42)
C = torch.rand((len(stoi), embed_dim)) # N, E
embeds = C[X] # N, E
